# Metaclasses

A class defines the rules for objects such as attributes, operations,
etc. But classes are objects at the same time. Who creates them?

The responsible of creating classes are the **metaclasses**. They define
the rules for a class. 

In [1]:
# Let's create a class
class Test:
    pass

print(Test)  # 'Test' is an object
print(Test())  # 'Test() is an instance of Test, and it's an object too

<class '__main__.Test'>


When can use **type()** to get the class of every instance. For example:

In [2]:
print(type(2))  # class 'int'
print(type("Hello"))  # class 'str'

# Functions are objects too, so we can get their types too
def f():
    pass
print(type(f))

# And for an instance of 'Test'
print(type(Test()))  # class 'Test'

<class 'int'>
<class 'str'>
<class 'function'>
<class '__main__.Test'>


Since classes are objects too, we are able to print their type too:

In [3]:
type(Test)  # class 'type'

type

This is because **type** is a metaclass, and it is responsible of creating classes. Let's see how we can create a class using *type*:

In [4]:
# Let's create a similar class using type's constructor
Test2 = type("Test2", (), {})  # type([name of the class], [parent classes], [attributes and methods])

In [5]:
# Now we are able to print the class type and to create instances of it
print(type(Test2))
print(type(Test2()))

<class 'type'>
<class '__main__.Test2'>


In [14]:
# Let's see how to add attributes and methods to that class

# Let's first define a method. It just prints 'Hello' and the name 
# attribute of the class 
def say_hello(self):
    print(f"Hello {self.name}!")

# Now let's call the type's constructor and give a value to the 'name'
# attribute and say that the method 'say_hello' is the function above
Test3 = type("Test3", (), {"name": "It's Test3!", "say_hello": say_hello})

# Now, we can access the class attributes and the methods as always
t = Test3()
print(t.name)
t.say_hello()

# We can add new attributes too
t.x = 5
print(t.x)

It's Test3!
Hello It's Test3!!
5


In [13]:
# We can do inheritance using this way of creating classes too.

# Let's say we have a parent class called 'Foo'
class Foo:
    
    def say_foo(self):
        print("Foo")
        
# And we want to create our class 'Test3' deriving it from Foo
Test3 = type("Test3", (Foo,), {"name": "It's Test3!", "say_hello": say_hello})

# Now, we can access the class attributes and the methods as before
t = Test3()
print(t.name)
t.say_hello()

# But we can also call 'say_foo'
t.say_foo()

It's Test3!
Hello It's Test3!!
Foo


Now we've seen how type() works as a metaclass for classes, we're going to see how to create our own metaclasses too:

In [21]:
# We create out metaclass deriving it from type
class Meta(
    type
):  # since this inheritance is implicit, we could have also said 'class Meta:'
    def __new__(self, class_name, bases, attribs):
        # This method is called when creating a new instance of class 'cls'.
        # After that, the __init__ method of 'cls' initializes the class.
        # Parameters are the same as the ones from type's constructor.

        # Let's call the type's constructor.
        print("Calling type's constructor from Meta.__new__()")
        print(
            f"Creating a class called {class_name} derived from "
            f"{bases} and with attributes {attribs}."
        )
        return type(class_name, bases, attribs)


# Now let's create a class that uses our metaclass 'Meta' instead of type.
# Actually, there's no huge difference due to the way we've defined __new__()
# in Meta.
class Dog(metaclass=Meta):
    animal = "dog"
    
    def say_hi(self):
        print(f"Hi, I'm {self.animal}")


# Let's create another class using the metaclass type to see the x.difference
# (which will be two print() statments)
class Cat:
    animal = "cat"
    
    def say_hi(self):
        print(f"Hi, I'm {self.animal}")


# Let's create and instance of Dog. We can see here that the __new__()
# method from Meta was called.
d = Dog()
d.say_hi()

# This doesn't happen when creating a Cat instance since it's built using
# type instead of Meta
c = Cat()
c.say_hi()


Calling type's constructor from Meta.__new__()
Creating a class called Dog derived from () and with attributes {'__module__': '__main__', '__qualname__': 'Dog', 'animal': 'dog', 'say_hi': <function Dog.say_hi at 0x000001691D2BDF70>}.
Hi, I'm dog
Hi, I'm cat


Knowing this, we can use metaclasses to modify a class creation. 

For example (a toy example), we could modify the 'animal' attribute.

In [28]:
# We create out metaclass deriving it from type
class Meta(
    type
):  # since this inheritance is implicit, we could have also said 'class Meta:'
    def __new__(self, class_name, bases, attribs):
        # This method is called when creating a new instance of class 'cls'.
        # After that, the __init__ method of 'cls' initializes the class.
        # Parameters are the same as the ones from type's constructor.

        
        # Let's modify the 'animal' attribute of 'cls'. It will always
        # be 'pitbull' (it's just an example, you might not want to do this)
        new_attribs = {}
        for attribute_name, attribute_values in attribs.items():
            if attribute_name.startswith("__"):
                new_attribs[attribute_name] = attribute_values
            else:
                if attribute_name == "animal":
                    attribute_values = "pitbull"
                new_attribs[attribute_name] = attribute_values
        
        # Let's call the type's constructor.
        print("Calling type's constructor from Meta.__new__()")
        print(
            f"Creating a class called {class_name} derived from "
            f"{bases} and with modified attributes {new_attribs}."
        )
        return type(class_name, bases, new_attribs)


# Now let's create a class that uses our metaclass 'Meta' instead of type.
# Actually, there's no huge difference due to the way we've defined __new__()
# in Meta.
class Dog(metaclass=Meta):
    animal = "dog"
    
    def say_hi(self):
        print(f"Hi, I'm {self.animal}")


# Let's create another class using the metaclass type to see the x.difference
# (which will be two print() statments)
class Cat:
    animal = "cat"
    
    def say_hi(self):
        print(f"Hi, I'm {self.animal}")


# Let's create and instance of Dog. We can see here that the __new__()
# method from Meta was called.
d = Dog()
d.say_hi()  # Now the dog's is always a pitbull

# This doesn't happen when creating a Cat instance since it's built using
# type instead of Meta
c = Cat()
c.say_hi()


Calling type's constructor from Meta.__new__()
Creating a class called Dog derived from () and with modified attributes {'__module__': '__main__', '__qualname__': 'Dog', 'animal': 'pitbull', 'say_hi': <function Dog.say_hi at 0x000001691D2BDCA0>}.
Hi, I'm pitbull
Hi, I'm cat


Actually, metaclasses are not usually used. Some applications could be:
- Using the 'Singleton' design pattern (if you consider it a pattern and not an antipattern).